# Visualisation of Thermodynamic Quantities

One of the first steps for determining a simulation has proceeded as expected,
particularly those that are supposed to be at equilibrium,
is to check the thermodynamic quantitites.
For simultions at equilibrium,
the thermodynamics should remain relatively constant over the course of the simulation.

This provides an interactive method of investigating 
all the values to check that they are as expected.

In [1]:
from pathlib import Path

import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt

from sdanalysis.util import get_filename_vars

In [2]:
data_dir = "../data/simulations/trimer/output/"
dataset = {}
for file in Path(data_dir).glob("thermo-*log"):
    temperature, pressure, _ = get_filename_vars(file.name)
    dataset.setdefault(pressure, {})
    dataset[pressure][temperature] = file

In [3]:
discard_quantities = ["timestep", "lz", "xz", "yz", "N"]
with open(file) as src:
    thermo_quantities = [c.strip() for c in src.readline().split('\t')]
    thermo_quantities = [c for c in thermo_quantities if c not in discard_quantities]
    thermo_quantities = [c for c in thermo_quantities if "rigid_center" not in c]

In [4]:
pressure = widgets.ToggleButtons(description="Pressure", options=list(dataset.keys()))
temperature = widgets.ToggleButtons(description="Temperature", options=dataset.get(pressure.value))
quantities = widgets.ToggleButtons(description="Quantity", options=thermo_quantities)

def update_temperatures(change):
    temperature.options = dataset.get(change.new)

pressure.observe(update_temperatures, names='value')


def plot_figure(filename, quantity):
    df = pd.read_csv(filename, sep='\t', index_col='timestep', usecols=["timestep", quantity])
    df = df[~df.index.duplicated(keep='first')].reset_index()
    df.index = pd.to_timedelta(df.timestep)
    df = df.resample('10ms').agg(['mean', 'std'])
    plt.errorbar(df.index.astype(int), df[quantity]['mean'], df[quantity]['std'], ecolor='k', elinewidth=1)
    
p = widgets.interactive(plot_figure, filename=temperature, quantity=quantities)

display(pressure, p)

ToggleButtons(description='Pressure', options=('13.50', '1.00'), value='13.50')

interactive(children=(ToggleButtons(description='Temperature', options={'1.25': PosixPath('../data/simulations…